In [8]:
# importing the required libraries
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

ModuleNotFoundError: No module named 'surprise'

In [2]:
# importing the datasets
books = pd.read_csv('F:\Data Science\Menternships\Movie Recommender System\Data - Hybrid Filtering\FinalData.csv')
books

,book_id,authors,title,Genres
0,1,Suzanne Collins,"The Hunger Games (The Hunger Games, #1)",SciFi;Drama
1,2,"J.K. Rowling, Mary GrandPré",Harry Potter and the Sorcerer's Stone (Harry P...,Fantasy;Young-Age
2,3,Stephenie Meyer,"Twilight (Twilight, #1)",Fantasy
3,4,Harper Lee,To Kill a Mockingbird,Self-Help;Drama
4,5,F. Scott Fitzgerald,The Great Gatsby,Drama
...,...,...,...,...
994,995,Michelle Hodkin,"The Unbecoming of Mara Dyer (Mara Dyer, #1)",Fiction
995,996,Janet Evanovich,"Three to Get Deadly (Stephanie Plum, #3)",Crime
996,997,Emmuska Orczy,The Scarlet Pimpernel,History;Fiction
997,998,"Jon Stone, Michael J. Smollin",The Monster at the End of this Book,Fiction;Kids


In [3]:
ratings = pd.read_csv('F:/Data Science/Menternships/Movie Recommender System/Data - Hybrid Filtering/ratings.csv')
ratings

,user_id,book_id,rating
0,1,258,5
1,2,260,5
2,2,26,4
3,2,315,3
4,2,33,4
...,...,...,...
3133539,49925,330,5
3133540,49925,510,5
3133541,49925,528,4
3133542,49925,722,4


In [4]:
df = pd.merge(books, ratings, on='book_id')
df

,book_id,authors,title,Genres,user_id,rating
0,1,Suzanne Collins,"The Hunger Games (The Hunger Games, #1)",SciFi;Drama,2886,5
1,1,Suzanne Collins,"The Hunger Games (The Hunger Games, #1)",SciFi;Drama,6158,5
2,1,Suzanne Collins,"The Hunger Games (The Hunger Games, #1)",SciFi;Drama,3991,4
3,1,Suzanne Collins,"The Hunger Games (The Hunger Games, #1)",SciFi;Drama,5281,5
4,1,Suzanne Collins,"The Hunger Games (The Hunger Games, #1)",SciFi;Drama,5721,5
...,...,...,...,...,...,...
3133539,999,Judy Blume,"Tales of a Fourth Grade Nothing (Fudge, #1)",Kids,12791,5
3133540,999,Judy Blume,"Tales of a Fourth Grade Nothing (Fudge, #1)",Kids,41773,5
3133541,999,Judy Blume,"Tales of a Fourth Grade Nothing (Fudge, #1)",Kids,20533,5
3133542,999,Judy Blume,"Tales of a Fourth Grade Nothing (Fudge, #1)",Kids,20967,5


Colaborative filtering and Content based filtering combined to get a Hybrid model

In [5]:
# Colaborative Filtering
# first create a pivot table for the data
ratings_df = pd.pivot_table(df, index='user_id', columns='book_id', values='rating')

In [7]:
# then calculate the similarity between the users
user_similarity = cosine_similarity(ratings_df.fillna(0))

MemoryError: Unable to allocate 21.3 GiB for an array with shape (53417, 53417) and data type float64

In [25]:
df_mean_ratings = df.groupby(['book_id','authors','title','Genres'])['rating'].mean().reset_index()

In [26]:
vectoriser = TfidfVectorizer()
books_vec = vectoriser.fit_transform(df_mean_ratings['Genres'])

In [27]:
cosine_sim = cosine_similarity(books_vec)

In [31]:
def rec_books(title, cosine_sim=cosine_sim, book_ratings=df_mean_ratings):      #define the function
    index=book_ratings[book_ratings['title']==title].index[0]                   #get the index of book that matches the title
    sim_scores = list(enumerate(cosine_sim[index]))                             #get the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x:x[1], reverse=True)            #sorting the movies
    sim_scores = sim_scores[1:11]                                               #get the top 10 similar books
    book_indices = [i[0] for i in sim_scores]                                   #gets the book index number
    recommended_books = book_ratings.iloc[book_indices][['title','rating']]     #gets book recommendations
    return recommended_books

In [33]:
rec_books('The Hunger Games (The Hunger Games, #1)')

,title,rating
16,"Catching Fire (The Hunger Games, #2)",4.133422
598,The Restaurant at the End of the Universe (Hit...,4.135909
716,The Hunger Games: Official Illustrated Movie C...,4.445946
69,"Ender's Game (Ender's Saga, #1)",4.258221
728,"Hyperion (Hyperion Cantos, #1)",4.136334
741,"Wool Omnibus (Silo, #1)",4.147078
750,"Life, the Universe and Everything (Hitchhiker'...",4.081731
759,"Ender's Shadow (Ender's Shadow, #1)",4.203125
788,Foundation and Empire (Foundation #2),4.127296
806,The Circle,3.429231
